In [1]:
import gc
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from datetime import datetime, timedelta, time

In [2]:
%%time

train = pd.read_parquet('/kaggle/input/otto-full-optimized-memory-footprint/train.parquet')
train['time'] = pd.to_datetime(train['ts']*1e9)

cutoff_w1 = train.time.min() + timedelta(days = 6, hours = 23, minutes = 59, seconds = 59)
print(cutoff_w1)

train_w1 = train[train.time <= cutoff_w1]
train_w1 = train_w1.reset_index(drop = True)
gc.collect()

2022-08-07 21:59:59
CPU times: user 41.4 s, sys: 27 s, total: 1min 8s
Wall time: 1min 13s


0

In [3]:
#drop one_event_sessions
one_event_df = train_w1.groupby('session', as_index = False).count()
one_event_df = one_event_df[one_event_df['aid']==1]
one_sessions =  one_event_df['session']
del one_event_df
train_w1 = train_w1[~train_w1['session'].isin(one_sessions)]
gc.collect()

36

In [4]:
%%time
train_w1_list = []
label_w1 = []

for grp in tqdm(train_w1.groupby('session')):
    cutoff = np.random.randint(1, grp[1].shape[0]) 
    train_w1_list.append(grp[1].iloc[:cutoff])
    label_w1.append(grp[1].iloc[cutoff:])
    gc.collect()

train_w1 = pd.concat(train_w1_list).reset_index(drop=True)
label_w1 = pd.concat(label_w1).reset_index(drop=True)

del train_w1_list

train_w1.to_parquet('train_w1.parquet')
label_w1.to_parquet('label_w1.parquet')

  0%|          | 411/4441142 [13:41<2465:57:54,  2.00s/it]


KeyboardInterrupt: 